## This is a JupyterLab Python script to find transitions from a Tanabe-Sugano diagram that are fitting your measured wavelengths
Author: Sebastian Wahl

#### Only Execute Cell (Import the modules and define some functions)

In [1]:
import pandas as pd
import numpy as np
import os
import statistics 
from IPython.display import display_html
from IPython.display import Javascript

#function to transfrom wavelength to wavenumber
def nm2cm(x):
    return 1/x/100*10**9

#function to transfrom wavelength to frequency
def nm2s(x):
    return ((299792458*10**9)/x)/10**13

#function to fit two points
def fitlin(dflow,dfup,indexlow,indexup):
    slope = (dfup - dflow) / (indexup - indexlow)
    axis = dfup - slope * indexup
    return slope, axis

#function to display to DataFrames next to each other
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

#### INPUT: wavelengths, name of sample, number of electrons, TS-diagram to use, field to apply

In [2]:
# Input lists of wavelengths

sample_wavelength = [522,557,605,632,680,878]


# select which dataset you want to use for the following calculations
ny_select=sample_wavelength
# Select the name of the output .txt file 
pathname='sample_wavelength'

#number of d-electrons
num_electrons = 6

#Use Tanabe-Sugano diagram 
d = 6

#select the field as 'td' or 'oh'
field= 'oh'

#select if strong field (1) or not (0)
strong_field=1

#### Only Execute Cell: Overview of Wavelengths, Wavenumbers, Frequencies and all possible Transitions

In [3]:
#creates DataFrame with wavelength, wavenumbers and frequencies
input_values = pd.DataFrame(columns = ['Wavelength','Wavenumber','Frequency'])
input_values['Wavelength'] = ny_select
input_values['Wavenumber'] = nm2cm(input_values['Wavelength'])
input_values['Frequency'] = nm2s(input_values['Wavelength'])
input_values = input_values.round({'Wavelength':0, 'Wavenumber':0, 'Frequency':2})

#imports the corresponding Tanabe-Sugano diagram
TSfile = 'C:folder/TSdiags/TS-d'+str(d)+'.csv'
dfts = pd.read_csv(TSfile)
dfts.columns = dfts.columns.str.strip().str.lower().str.replace('(', '').str.replace(')', '').str.replace('.', '_')
all_transitions=list(dfts)

transitions_overview = pd.DataFrame(data=all_transitions, columns=["Transition"])
display_side_by_side(input_values, transitions_overview)

,Wavelength,Wavenumber,Frequency
0,60.0,167369.0,501.76
1,306.0,32634.0,97.83
2,412.0,24263.0,72.74
3,438.0,22843.0,68.48
4,475.0,21035.0,63.06
5,541.0,18499.0,55.46
6,666.0,15014.0,45.01
7,826.0,12099.0,36.27
8,1267.0,7891.0,23.66
9,1328.0,7528.0,22.57


#### INPUT: Select 2 or 3 wavelength (order by decreasing wavelength) by index, and assign the most probable transitions by index

In [4]:
selected_bands=[6,2,4]

selected_transitions=[16,17,2]

#### Only Execute Cell

In [5]:
#creates a list with the selected wavenumbers
wn = [] #empty list
for i in range(len(selected_bands)):
    wn.append(input_values.loc[selected_bands[i],'Wavenumber'])

#creates a list with the selected transitions
transition=[] #empty list
for i in range(len(selected_transitions)):
    transition.append(all_transitions[selected_transitions[i]])    
    
#creates a list with the the ratios band1/band0 and band2/band1
ratio=[]
for i in range(len(wn)-1):
    ratio.append(wn[i+1]/wn[i])    
    
display("Wavenumbers selected:",wn)
display("Ratios:",ratio)
display("Transitions selected:",transition)

'Wavenumbers selected:'

[15014.0, 24263.0, 21035.0]

'Ratios:'

[1.6160250432929266, 0.8669579194658533]

'Transitions selected:'

['1t1gi', '1t2gi', '5egd']

#### INPUT: Select the ratio you want to use (0 or 1)

In [6]:
selected_ratio = 0

#### Only execute cell: calculation of approximated Delta/B, E/B and B values

In [7]:
#if strong field, the Tanabe-Sugano diagram will be read from the end
if strong_field==1:
    strong_field_index = -1
else:
    strong_field_index = 0

#calculates the ratio between the selected transitions
delta_B_ratio =  dfts[transition[selected_ratio+1]] / dfts[transition[selected_ratio]]
deltas2 = []
for i in range(len(delta_B_ratio)):
    deltas2.append(abs(delta_B_ratio[i]-ratio[selected_ratio]))    
    
#calculates the minimum deviation from the calculated ratios    
r_min = []
for i in range(len(deltas2)):
    if  deltas2[i] < min(deltas2[i - 1], deltas2[(i + 1) % len(deltas2)]):
        r_min.append(deltas2[i])

#       
#if more than 2 minima were found, put the minimum you want to use into strong_field_index below and uncomment
#
#strong_field_index = 1
#
#

#outputs the index of the minimum deviation
index1 = deltas2.index(r_min[strong_field_index])

deltas = []

deltas= pd.DataFrame(columns=['Delta_B_ratio'])
deltas['Delta_B_ratio']=delta_B_ratio

deltas['findmin']=deltas2

#rearranges the upper and lower index correctly
if abs(deltas.at[index1,'findmin']-deltas.at[index1+1,'findmin']) > abs(deltas.at[index1,'findmin']-deltas.at[index1-1,'findmin']):
    index2 = index1-1
else:
    index2 = index1+1

if index1 < index2:
    lower_index = index1
    upper_index = index2
else:
    lower_index = index2
    upper_index = index1

#linear fit between the upper and lower index
slope = (deltas['Delta_B_ratio'].loc[upper_index]-deltas['Delta_B_ratio'].loc[lower_index])/(upper_index - lower_index)
axis = deltas['Delta_B_ratio'].loc[upper_index]-(slope*upper_index)

#approximates the index based on linear fit
approx_index = (ratio[selected_ratio]-axis)/slope
print('approximated index : ',approx_index)

#approximates the Delta/B value based on the indices, by linear fit between 2 points
delta_B_lower = dfts.at[lower_index, 'delta/b']
delta_B_upper = dfts.at[upper_index, 'delta/b']
slope2 = (delta_B_upper - delta_B_lower)/ (upper_index - lower_index)
axis2 = delta_B_upper - slope2 * upper_index
delta_B_approx = approx_index * slope2 + axis2
print('Δ/B_approx =',delta_B_approx)

#approximates the E/B value based on the indices, by linear fit between 2 points
EB = []
for i in range(len(wn)):
    delta_EB_lower = dfts.at[lower_index, transition[i]]
    delta_EB_upper = dfts.at[upper_index, transition[i]]
    slopeEB = (delta_EB_upper - delta_EB_lower)/ (upper_index - lower_index)
    axisEB = delta_EB_upper - slopeEB * upper_index
    delta_EB_approx = approx_index * slopeEB + axisEB
    EB.append(delta_EB_approx)
    print('E/B('+transition[i]+') =',delta_EB_approx)

#calculates the B value based on the previous calculations
B=[]
for i in range(len(wn)):
    B.append(wn[i]/EB[i])
    print('B('+transition[i]+') =', B[i])

display("values in r_min. If more than 2, insert r_min manually above", r_min)

approximated index :  38.037535712051444
Δ/B_approx = 21.131964275030167
E/B(1t1gi) = 18.6093729178131
E/B(1t2gi) = 30.073004560909055
E/B(5egd) = 26.00814453094968
B(1t1gi) = 806.7977393063272
B(1t2gi) = 806.8033225898122
B(5egd) = 808.7851086404246


'values in r_min. If more than 2, insert r_min manually above'

[0.005947389775228018, 0.00040460636653083526]

#### INPUT: Chose "0" if you want the average of all B or "1" to use only 2 values

In [8]:
B_own = 1

#### Only Execute Cell: Overview of calculated values for comparison

In [9]:
#calculates average of B
B_aver = (B[selected_ratio]+B[selected_ratio+1])/2
if B_own == 0:
    B_aver = statistics.mean(B)
else:
    pass

#calculates Delta_o or Delta_t based on previous input
delta=delta_B_approx*B_aver

if field == 'td':
    delta_o=delta*9/4
    delta_t=delta
elif field == 'oh':    
    delta_o=delta

#print the calculated values
print('')
print('B average =',round(B_aver,1))
print('')
print('Δ/B = ', round(delta_B_approx,1))
print('')
print('Δo = ', round(delta_o,0))
if field == 'td':
    print('Δt = ', round(delta_t,0))
else:
    pass

print('')  
print(input_values.iloc[selected_bands,:])
print('')

calculated_lambda=[]
for i in range(len(wn)):
    calculated_lambda.append(nm2cm(EB[i]*B_aver))
    print('λ_calc'+str(i)+' =',round(calculated_lambda[i],1),'\tν_calc'+str(i)+' =',round(EB[i]*B_aver,1),'\t\ttransition =', transition[i])



B average = 806.8

Δ/B =  21.1

Δo =  17049.0

   Wavelength  Wavenumber  Frequency
6       666.0     15014.0      45.01
2       412.0     24263.0      72.74
4       475.0     21035.0      63.06

λ_calc0 = 666.0 	ν_calc0 = 15014.1 		transition = 1t1gi
λ_calc1 = 412.2 	ν_calc1 = 24262.9 		transition = 1t2gi
λ_calc2 = 476.6 	ν_calc2 = 20983.4 		transition = 5egd


#### Only execute cell: Possible transitions at the calculated Δ/B and their E/B values (ignore error given for dividing by zero)
Here you need to manually compare the calculated values to the measured and find the best fitting yourself

In [25]:
list2=[]
list2.append((None,None,None,None))
for i in range(1,len(dfts.columns)):
    EB_calc = fitlin(dfts.iloc[lower_index, i],dfts.iloc[upper_index, i], lower_index, upper_index)
    EB_approx = approx_index * EB_calc[0]+EB_calc[1]
    wn_approx = EB_approx * B_aver
    wl_approx = nm2cm(EB_approx * B_aver)
    list2.append((dfts.columns[i],round(EB_approx,2), round(wl_approx,0), round(wn_approx,0)))
df2 = pd.DataFrame(data=list2,columns =['transition', 'E/B', 'wavelength', 'wavenumber'])
df3=df2.sort_values(by=['E/B'])
display_side_by_side(df3,input_values)

D:\Programme\anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


,transition,E/B,wavelength,wavenumber
15,1a1gi,0.00,inf,0.0
1,5t2gd,4.88,2542.000000,3934.0
3,3t1gh,8.99,1379.000000,7251.0
4,3t2gh,14.67,845.000000,11834.0
16,1t1gi,18.61,666.000000,15014.0
2,5egd,26.01,477.000000,20983.0
6,3t1gh_1,27.70,447.000000,22350.0
8,3t2gf,29.16,425.000000,23528.0
17,1t2gi,30.07,412.000000,24263.0
5,3egh,31.74,390.000000,25612.0


#### Only execute cell: Latex output

In [26]:
print("B average = " + str(int(round(B_aver,0))) + "~\si{\per\centi\metre}")
print("Delta/B = " + str(round(delta_B_approx,1))+ "\n")
for i in reversed(range(len(input_values))):
    print(int(input_values.loc[i,"Wavelength"]),'\t&',int(input_values.loc[i,"Wavenumber"]),'\t&','\t&','\t \\\\')

B average = 807~\si{\per\centi\metre}
Delta/B = 21.1

1667 	& 5999 	& 	& 	 \\
1537 	& 6505 	& 	& 	 \\
1372 	& 7286 	& 	& 	 \\
1328 	& 7528 	& 	& 	 \\
1267 	& 7891 	& 	& 	 \\
826 	& 12099 	& 	& 	 \\
666 	& 15014 	& 	& 	 \\
541 	& 18499 	& 	& 	 \\
475 	& 21035 	& 	& 	 \\
438 	& 22843 	& 	& 	 \\
412 	& 24263 	& 	& 	 \\
306 	& 32634 	& 	& 	 \\
60 	& 167369 	& 	& 	 \\


### Export results into a .txt file, change name of textfile before in 'fpath'. Calculated transitions are exported into a .csv file

In [28]:
#change fpath to the directory where you want to have your output
fpath='C:folder/TS-Analyses/'+pathname

file = open(fpath+'.txt','w+')
file.write("input data: \n\n")
file.write("Number of d-electrons: "+str(num_electrons)+"\n")
file.write("Tanabe-Sugano diagram used: d"+str(d)+"\n\n")


for i in range(len(input_values)):
    file.write("lambda"+str(i)+" = " + str(input_values.loc[i,"Wavelength"]) + "    wavenumber"+str(i)+" = " + str(input_values.loc[i,"Wavenumber"]) + "\n")

file.write('\n')

file.write("Selected bands: "+str(selected_bands)+"\n")
file.write("Selected transitions: "+str(selected_transitions)+"\n")

file.write('\n')

file.write("Calculated values:\n")
file.write("\n B average = " + str(int(round(B_aver,0))) + "~\si{\per\centi\metre}")
file.write("\n Delta/B = " + str(round(delta_B_approx,1)))
file.write("\n Delta_o = " + str(round(delta_o,0))+ "~\si{\per\centi\metre}")
file.write("\n")

if field == 'td':
    file.write("\n Delta_t = " + str(round(delta_t,0))+ "~\si{\per\centi\metre}")
else:
    pass


file.close()

df3.to_csv(fpath+'-calc.csv')